# Spark Session

In [1]:
from pyspark.sql import SparkSession

# Spark session & context
spark = (SparkSession
         .builder
         .master("spark://spark:7077")
         .appName("read-postgres")
         # Add JARS
         #.config('spark.jars.packages', 'org.postgresql:postgresql:9.4.1207,com.amazonaws:aws-java-sdk-bundle:1.11.315,org.apache.hadoop:hadoop-aws:3.2.0') 
         .config('spark.jars.packages', 'org.postgresql:postgresql:9.4.1207,com.amazonaws:aws-java-sdk-bundle:1.11.315,org.apache.hadoop:hadoop-aws:3.2.0')
         .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
         .config("spark.hadoop.fs.s3a.access.key", "minio")
         .config("spark.hadoop.fs.s3a.secret.key", "miniosecret")
         .config("spark.hadoop.fs.s3a.path.style.access", True)
         .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
         .enableHiveSupport()
         .getOrCreate())

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.postgresql#postgresql added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9fa00718-349b-499a-b912-ae161204b746;1.0
	confs: [default]
	found org.postgresql#postgresql;9.4.1207 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.315 in central
	found io.netty#netty-codec-http;4.1.17.Final in central
	found io.netty#netty-codec;4.1.17.Final in central
	found io.netty#netty-transport;4.1.17.Final in central
	found io.netty#netty-buffer;4.1.17.Final in central
	found io.netty#netty-common;4.1.17.Final in central
	found io.netty#netty-resolver;4.1.17.Final in central
	found io.netty#netty-handler;4.1.17.Final in central
	found org.apache.hadoop#hadoop-aws;3.2.0 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.375 in central


# Reading from PgSQL

In [15]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
+---------+



In [2]:
df_dags = (
    spark.read
    .format("jdbc")
    .option("url", "jdbc:postgresql://postgres/airflow")
    .option("driver", "org.postgresql.Driver")
    .option("dbtable", "public.dag")
    .option("user", "airflow")
    .option("password", "airflow")
    .load() )

df_dags.show()

+--------------------+---------+---------+---------+--------------------+------------+------------+--------------+---------+--------------------+-------+--------------------+------------+--------------------+-----------+
|              dag_id|is_paused|is_subdag|is_active|  last_scheduler_run|last_pickled|last_expired|scheduler_lock|pickle_id|             fileloc| owners|         description|default_view|   schedule_interval|root_dag_id|
+--------------------+---------+---------+---------+--------------------+------------+------------+--------------+---------+--------------------+-------+--------------------+------------+--------------------+-----------+
|      spark-postgres|     true|    false|     true|2021-12-29 17:27:...|        null|        null|          null|     null|/usr/local/airflo...|airflow|This DAG is a sam...|        null|{"type": "timedel...|       null|
|spark-hello-world...|     true|    false|     true|2021-12-29 17:27:...|        null|        null|          null|  

# Write in MinIO

In [3]:
df_dags.write.parquet("s3a://stage/dags")

21/12/29 17:28:00 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [13]:
spark.stop()